In [ ]:
# Import Required Packages
import pandas as pd

In [64]:
# Financial_statement ========= work in progress.....! =============


class IncomeStatement:
    def __init__(self, assumptions, historical_data):
        self.assumptions = assumptions
        self.historical_data = historical_data

    def calculate_revenue(self):
        # Calculate revenue based on the revenue growth rate
        revenue_growth = self.assumptions["Revenue Growth Rate"]
        last_year_revenue = self.historical_data["Revenue"].iloc[-1]
        projected_years = [
            self.historical_data["Year"].iloc[-1] + i for i in range(1, 6)
        ]
        projected_revenue = [
            last_year_revenue * (1 + revenue_growth) ** i for i in range(1, 6)
        ]

        return pd.DataFrame({"Year": projected_years, "Revenue": projected_revenue})

    def calculate_cogs(self):
        # Calculate COGS based on the COGS as % of Revenue assumption
        projected_cogs = [
            self.calculate_revenue()["Revenue"].iloc[i]
            * self.assumptions["COGS as % of Revenue"]
            for i in range(5)
        ]

        return pd.DataFrame(
            {
                "Year": self.calculate_revenue()["Year"],
                "Cost of Goods Sold (COGS)": projected_cogs,
            }
        )

    def calculate_gross_profit(self):
        revenue = self.calculate_revenue()["Revenue"]
        cogs = self.calculate_cogs()["Cost of Goods Sold (COGS)"]
        gross_profit = revenue - cogs

        return pd.DataFrame(
            {"Year": self.calculate_revenue()["Year"], "Gross Profit": gross_profit}
        )

    def calculate_sga_expenses(self):
        revenue = self.calculate_revenue()["Revenue"]
        sga_expenses = revenue * self.assumptions["SG&A as % of Sales"]

        return pd.DataFrame(
            {"Year": self.calculate_revenue()["Year"], "SG&A Expenses": sga_expenses}
        )

    def calculate_operating_income(self):
        gross_profit = self.calculate_gross_profit()["Gross Profit"]
        sga_expenses = self.calculate_sga_expenses()["SG&A Expenses"]
        operating_income = gross_profit - sga_expenses

        return pd.DataFrame(
            {
                "Year": self.calculate_revenue()["Year"],
                "Operating Income": operating_income,
            }
        )

    def calculate_interest_expense(self):
        net_debt = (
            self.historical_data["Total Liabilities"] - self.historical_data["Cash"]
        )
        interest_expense = net_debt * self.assumptions["LIBOR"]

        return pd.DataFrame(
            {
                "Year": self.calculate_revenue()["Year"],
                "Interest Expense": interest_expense,
            }
        )

    def calculate_net_income(self):
        operating_income = self.calculate_operating_income()["Operating Income"]
        interest_expense = self.calculate_interest_expense()["Interest Expense"]
        other_income_expense = self.historical_data["Other Income / (Expense)"].iloc[-1]
        taxes = (
            operating_income - interest_expense + other_income_expense
        ) * self.assumptions["Tax Rate"]
        net_income = operating_income - interest_expense + other_income_expense - taxes

        return pd.DataFrame(
            {"Year": self.calculate_revenue()["Year"], "Net Income": net_income}
        )

    def calculate_all_line_items(self):
        projected_revenue = self.calculate_revenue()
        projected_cogs = self.calculate_cogs()
        projected_gross_profit = self.calculate_gross_profit()
        projected_sga_expenses = self.calculate_sga_expenses()
        projected_operating_income = self.calculate_operating_income()
        projected_interest_expense = self.calculate_interest_expense()
        projected_net_income = self.calculate_net_income()

        return pd.DataFrame(
            {
                "Year": projected_revenue["Year"],
                "Revenue": projected_revenue["Revenue"],
                "Cost of Goods Sold (COGS)": projected_cogs[
                    "Cost of Goods Sold (COGS)"
                ],
                "Gross Profit": projected_gross_profit["Gross Profit"],
                "SG&A Expenses": projected_sga_expenses["SG&A Expenses"],
                "Operating Income": projected_operating_income["Operating Income"],
                "Interest Expense": projected_interest_expense["Interest Expense"],
                "Net Income": projected_net_income["Net Income"],
            }
        )


class BalanceSheet:
    def __init__(self, assumptions, historical_data):
        self.assumptions = assumptions
        self.historical_data = historical_data

    def calculate_inventory(self):
        # Calculate inventory based on the days inventory assumption
        days_inventory = self.assumptions["Days Inventory"]
        projected_inventory = [
            (self.historical_data["Cost of Goods Sold (COGS)"].iloc[i] / 365)
            * days_inventory
            for i in range(5)
        ]

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Inventory": projected_inventory,
            }
        )

    def calculate_accounts_receivable(self):
        days_accounts_receivable = self.assumptions["Days Accounts Receivable"]
        projected_accounts_receivable = [
            (self.historical_data["Revenue"].iloc[i] / 365) * days_accounts_receivable
            for i in range(5)
        ]

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Accounts Receivable": projected_accounts_receivable,
            }
        )

    def calculate_other_current_assets(self):
        other_current_assets = self.assumptions["Other Current Assets"]

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Other Current Assets": [other_current_assets] * 5,
            }
        )

    def calculate_total_current_assets(self):
        inventory = self.calculate_inventory()["Inventory"]
        accounts_receivable = self.calculate_accounts_receivable()[
            "Accounts Receivable"
        ]
        other_current_assets = self.calculate_other_current_assets()[
            "Other Current Assets"
        ]

        total_current_assets = inventory + accounts_receivable + other_current_assets

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Total Current Assets": total_current_assets,
            }
        )

    def calculate_net_ppe(self):
        gross_ppe = self.historical_data["Gross PP&E"]
        accumulated_depreciation = self.historical_data["Accumulated Depreciation"]
        net_ppe = gross_ppe - accumulated_depreciation

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Net PP&E": net_ppe,
            }
        )

    def calculate_goodwill(self):
        goodwill = self.historical_data["Goodwill"].iloc[-1]
        projected_goodwill = [goodwill] * 5

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Goodwill": projected_goodwill,
            }
        )

    def calculate_other_assets(self):
        other_assets = self.assumptions["Other Assets"]

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Other Assets": [other_assets] * 5,
            }
        )

    def calculate_total_assets(self):
        total_current_assets = self.calculate_total_current_assets()[
            "Total Current Assets"
        ]
        net_ppe = self.calculate_net_ppe()["Net PP&E"]
        goodwill = self.calculate_goodwill()["Goodwill"]
        other_assets = self.calculate_other_assets()["Other Assets"]

        total_assets = total_current_assets + net_ppe + goodwill + other_assets

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Total Assets": total_assets,
            }
        )

    def calculate_accounts_payable(self):
        days_payable = self.assumptions["Days Payable"]
        projected_accounts_payable = [
            (self.historical_data["Cost of Goods Sold (COGS)"].iloc[i] / 365)
            * days_payable
            for i in range(5)
        ]

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Accounts Payable": projected_accounts_payable,
            }
        )

    def calculate_accrued_liabilities(self):
        accrued_liabilities_as_percentage_of_cogs = self.assumptions[
            "Accrued Liabilities as % of COGS"
        ]
        projected_accrued_liabilities = [
            self.historical_data["Cost of Goods Sold (COGS)"].iloc[i]
            * accrued_liabilities_as_percentage_of_cogs
            for i in range(5)
        ]

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Accrued Liabilities": projected_accrued_liabilities,
            }
        )

    def calculate_other_current_liabilities(self):
        other_current_liabilities_as_percentage_of_cogs = self.assumptions[
            "Other Current Liabilities as % of COGS"
        ]
        projected_other_current_liabilities = [
            self.historical_data["Cost of Goods Sold (COGS)"].iloc[i]
            * other_current_liabilities_as_percentage_of_cogs
            for i in range(5)
        ]

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Other Current Liabilities": projected_other_current_liabilities,
            }
        )

    def calculate_total_current_liabilities(self):
        accounts_payable = self.calculate_accounts_payable()["Accounts Payable"]
        accrued_liabilities = self.calculate_accrued_liabilities()[
            "Accrued Liabilities"
        ]
        other_current_liabilities = self.calculate_other_current_liabilities()[
            "Other Current Liabilities"
        ]

        total_current_liabilities = (
            accounts_payable + accrued_liabilities + other_current_liabilities
        )

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Total Current Liabilities": total_current_liabilities,
            }
        )

    def calculate_total_liabilities(self):
        total_current_liabilities = self.calculate_total_current_liabilities()[
            "Total Current Liabilities"
        ]
        other_liabilities = self.assumptions["Other Liabilities"]

        total_liabilities = total_current_liabilities + other_liabilities

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Total Liabilities": total_liabilities,
            }
        )

    def calculate_common_stock(self):
        common_stock = self.assumptions["Common Stock"]
        projected_common_stock = [common_stock] * 5

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Common Stock": projected_common_stock,
            }
        )

    def calculate_total_shareholders_equity(self):
        common_stock = self.calculate_common_stock()["Common Stock"]
        retained_earnings = self.historical_data["Retained Earnings"]
        total_shareholders_equity = common_stock + retained_earnings

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Total Shareholders Equity": total_shareholders_equity,
            }
        )

    def calculate_total_liabilities_and_equity(self):
        total_liabilities = self.calculate_total_liabilities()["Total Liabilities"]
        total_shareholders_equity = self.calculate_total_shareholders_equity()[
            "Total Shareholders Equity"
        ]
        total_liabilities_and_equity = total_liabilities + total_shareholders_equity

        return pd.DataFrame(
            {
                "Year": self.historical_data["Year"].iloc[-1]
                + [i for i in range(1, 6)],
                "Total Liabilities and Equity": total_liabilities_and_equity,
            }
        )

    def calculate_all_line_items(self):
        projected_inventory = self.calculate_inventory()
        projected_accounts_receivable = self.calculate_accounts_receivable()
        projected_other_current_assets = self.calculate_other_current_assets()
        projected_total_current_assets = self.calculate_total_current_assets()
        projected_net_ppe = self.calculate_net_ppe()
        projected_goodwill = self.calculate_goodwill()
        projected_other_assets = self.calculate_other_assets()
        projected_total_assets = self.calculate_total_assets()
        projected_accounts_payable = self.calculate_accounts_payable()
        projected_accrued_liabilities = self.calculate_accrued_liabilities()
        projected_other_current_liabilities = self.calculate_other_current_liabilities()
        projected_total_current_liabilities = self.calculate_total_current_liabilities()
        projected_total_liabilities = self.calculate_total_liabilities()
        projected_common_stock = self.calculate_common_stock()
        projected_total_shareholders_equity = self.calculate_total_shareholders_equity()
        projected_total_liabilities_and_equity = (
            self.calculate_total_liabilities_and_equity()
        )

        return pd.DataFrame(
            {
                "Year": projected_inventory["Year"],
                "Inventory": projected_inventory["Inventory"],
                "Accounts Receivable": projected_accounts_receivable[
                    "Accounts Receivable"
                ],
                "Other Current Assets": projected_other_current_assets[
                    "Other Current Assets"
                ],
                "Total Current Assets": projected_total_current_assets[
                    "Total Current Assets"
                ],
                "Net PP&E": projected_net_ppe["Net PP&E"],
                "Goodwill": projected_goodwill["Goodwill"],
                "Other Assets": projected_other_assets["Other Assets"],
                "Total Assets": projected_total_assets["Total Assets"],
                "Accounts Payable": projected_accounts_payable["Accounts Payable"],
                "Accrued Liabilities": projected_accrued_liabilities[
                    "Accrued Liabilities"
                ],
                "Other Current Liabilities": projected_other_current_liabilities[
                    "Other Current Liabilities"
                ],
                "Total Current Liabilities": projected_total_current_liabilities[
                    "Total Current Liabilities"
                ],
                "Total Liabilities": projected_total_liabilities["Total Liabilities"],
                "Common Stock": projected_common_stock["Common Stock"],
                "Total Shareholders Equity": projected_total_shareholders_equity[
                    "Total Shareholders Equity"
                ],
                "Total Liabilities and Equity": projected_total_liabilities_and_equity[
                    "Total Liabilities and Equity"
                ],
            }
        )


# class CashFlow:
#     def __init__(self, assumptions, historical_data, projected_income_statement):
#         self.assumptions = assumptions
#         self.historical_data = historical_data
#         self.projected_income_statement = projected_income_statement

#     def calculate_cash_flow_from_operations(self):
#         # Calculate cash flow from operations based on the net income and other assumptions
#         net_income = self.projected_income_statement["Net Income"]
#         depreciation_amortization = self.historical_data["Depreciation and Amortization"].iloc[-1]
#         projected_cash_flow_from_operations = net_income + depreciation_amortization

#         return pd.DataFrame({
#             "Year": self.projected_income_statement["Year"],
#             "Cash Flow from Operations": projected_cash_flow_from_operations
#         })

#     def calculate_capital_expenditures(self):
#         capex_as_percentage_of_sales = self.assumptions["Capex as % of Sales"]
#         projected_revenue = self.projected_income_statement["Revenue"]
#         projected_capital_expenditures = projected_revenue * capex_as_percentage_of_sales

#         return pd.DataFrame({
#             "Year": self.projected_income_statement["Year"],
#             "Capital Expenditures": projected_capital_expenditures
#         })

#     def calculate_asset_disposition(self):
#         asset_disposition = self.assumptions["Asset Disposition"]
#         projected_asset_disposition = [asset_disposition] * 5

#         return pd.DataFrame({
#             "Year": self.projected_income_statement["Year"],
#             "Asset Disposition": projected_asset_disposition
#         })

#     def calculate_cash_flow_from_investing(self):
#         capital_expenditures = self.calculate_capital_expenditures()["Capital Expenditures"]
#         asset_disposition = self.calculate_asset_disposition()["Asset Disposition"]
#         cash_flow_from_investing = capital_expenditures - asset_disposition

#         return pd.DataFrame({
#             "Year": self.projected_income_statement["Year"],
#             "Cash Flow from Investing": cash_flow_from_investing
#         })

#     def calculate_change_in_unsecured_debt(self):
#         unsecured_debt_amortization = self.assumptions["Unsecured Debt Amortization"]
#         projected_change_in_unsecured_debt = [-unsecured_debt_amortization] * 5

#         return pd.DataFrame({
#             "Year": self.projected_income_statement["Year"],
#             "Change in Unsecured Debt": projected_change_in_unsecured_debt
#         })

#     def calculate_cash_flow_from_financing(self):
#         cash_flow_from_investing = self.calculate_cash_flow_from_investing()["Cash Flow from Investing"]
#         change_in_unsecured_debt = self.calculate_change_in_unsecured_debt()["Change in Unsecured Debt"]
#         cash_flow_from_financing = cash_flow_from_investing + change_in_unsecured_debt

#         return pd.DataFrame({
#             "Year": self.projected_income_statement["Year"],
#             "Cash Flow from Financing": cash_flow_from_financing
#         })

#     def calculate_net_cash_flow(self):
#         cash_flow_from_operations = self.calculate_cash_flow_from_operations()["Cash Flow from Operations"]
#         cash_flow_from_investing = self.calculate_cash_flow_from_investing()["Cash Flow from Investing"]
#         cash_flow_from_financing = self.calculate_cash_flow_from_financing()["Cash Flow from Financing"]
#         net_cash_flow = cash_flow_from_operations + cash_flow_from_investing + cash_flow_from_financing

#         return pd.DataFrame({
#             "Year": self.projected_income_statement["Year"],
#             "Net Cash Flow": net_cash_flow
#         })

#     def calculate_ending_cash_position(self):
#         beginning_cash_position = self.historical_data["Ending Cash Position"].iloc[-1]
#         net_cash_flow = self.calculate_net_cash_flow()["Net Cash Flow"]
#         ending_cash_position = beginning_cash_position + net_cash_flow

#         return pd.DataFrame({
#             "Year": self.projected_income_statement["Year"],
#             "Ending Cash Position": ending_cash_position
#         })

#     def calculate_all_line_items(self):
#         projected_cash_flow_from_operations = self.calculate_cash_flow_from_operations()
#         projected_cash_flow_from_investing = self.calculate_cash_flow_from_investing()
#         projected_cash_flow_from_financing = self.calculate_cash_flow_from_financing()
#         projected_net_cash_flow = self.calculate_net_cash_flow()
#         projected_ending_cash_position = self.calculate_ending_cash_position()

#         return pd.DataFrame({
#             "Year": projected_cash_flow_from_operations["Year"],
#             "Cash Flow from Operations": projected_cash_flow_from_operations["Cash Flow from Operations"],
#             "Capital Expenditures": projected_cash_flow_from_investing["Cash Flow from Investing"],
#             "Asset Disposition": projected_cash_flow_from_investing["Asset Disposition"],
#             "Cash Flow from Investing": projected_cash_flow_from_investing["Cash Flow from Investing"],
#             "Change in Unsecured Debt": projected_cash_flow_from_financing["Change in Unsecured Debt"],
#             "Cash Flow from Financing": projected_cash_flow_from_financing["Cash Flow from Financing"],
#             "Net Cash Flow": projected_net_cash_flow["Net Cash Flow"],
#             "Ending Cash Position": projected_ending_cash_position["Ending Cash Position"]
#         })

In [42]:
# reead in the  input data
assumptions = {
    # Your assumptions here
    "Revenue Growth Rate": 0.05,
    "COGS as % of Revenue": 0.4,
    "Depreciation as % of Gross PP&E": 0.02,
    "Amortization": 0,
    "SG&A as % of Sales": 0.3,
    "Other Income / (Expense)": 0,
    "Tax Rate": 0.4,
    "Days Accounts Receivable": 30,
    "Days Inventory": 45,
    "Other Current Assets": 1,
    "Other Assets": 0,
    "Capex as % of Sales": 0.05,
    "Asset Disposition": 0,
    "Days Payable": 50,
    "Accrued Liabilities as % of COGS": 0.03,
    "Other Current Liabilities as % of COGS": 0.02,
    "Other Liabilities": 2,
    "Common Stock": 10,
    "LIBOR": 0.01,
    "Interest earned on cash": 0.0063,
    "Revolver": 0.03,
    "Term Loan": 0.035,
    "Unsecured Debt": 0.12,
    "Term Loan Amortization": 20,
    "Unsecured Debt Amortization": 0,
}

historical_data = pd.DataFrame(
    {
        # Your historical data here
        "Year": [2012, 2013, 2014, 2015, 2016],
        "Revenue": [160, 170, 180, 190, 200],
        "% Growth": [None, 0.0625, 0.058823529, 0.055555556, 0.052631579],
        "Cost of Goods Sold (COGS)": [67.2, 70.55, 73.8, 77.9, 80],
        "COGS as % of Revenue": [0.42, 0.415, 0.41, 0.41, 0.4],
        "Gross Profit": [92.8, 99.45, 106.2, 112.1, 120],
        "Gross Profit Margin %": [0.58, 0.585, 0.59, 0.59, 0.6],
        "Depreciation": [5, 5, 5, 5, 5],
        "Amortization": [0, 0, 0, 0, 0],
        "SG&A Expenses": [48, 51, 54, 57, 60],
        "SG&A as % of Revenue": [0.3, 0.3, 0.3, 0.3, 0.3],
        "Operating Income / EBIT": [39.8, 43.45, 47.2, 50.1, 55],
        "Operating Income / EBIT Margin %": [
            0.24875,
            0.255588235,
            0.262222222,
            0.263684211,
            0.275,
        ],
        "EBITDA": [44.8, 48.45, 52.2, 55.1, 60],
        "EBITDA Margin %": [0.28, 0.285, 0.29, 0.29, 0.3],
        "EBITDA Growth": [None, 0.081473214, 0.077399381, 0.055555556, 0.08892922],
        "Interest Expense": [13, 18.01720978, 17.15867571, 16.40339295, 15.48661306],
        "Interest Income": [0.2, 0, 0, 0, 0],
        "Net Interest Expense": [
            12.8,
            18.01720978,
            17.15867571,
            16.40339295,
            15.48661306,
        ],
        "Other Income / (Expense)": [2, 0, -2, 1, 0],
        "Pretax Income": [29, 25.43279022, 28.04132429, 34.69660705, 39.51338694],
        "Taxes": [11.6, 10.17311609, 11.21652971, 13.87864282, 15.80535478],
        "Tax Rate": [0.4, 0.4, 0.4, 0.4, 0.4],
        "Net Income": [17.4, 15.25967413, 16.82479457, 20.81796423, 23.70803216],
        "Net Margin %": [0.10875, 0.089762789, 0.093471081, 0.109568233, 0.118540161],
        "Net Income Growth": [None, -0.123007234, 0.102565784, 0.23733839, 0.138825675],
        "Cash": [5, 0, 0, 0, 8.69704e-12],
        "Accounts Receivable": [12, 13, 14, 15, 16],
        "Inventory": [8, 8.5, 9, 9.5, 10],
        "Other Current Assets": [1, 1, 1, 1, 1],
        "Total Current Assets": [26, 22.5, 24, 25.5, 27],
        "Gross PP&E": [277.2, 287.2, 298.2, 310.2, 323.2],
        "Accumulated Depreciation": [25, 30, 35, 40, 45],
        "Net PP&E": [252.2, 257.2, 263.2, 270.2, 278.2],
        "Other Assets": [0, 0, 0, 0, 0],
        "Goodwill": [5, 5, 5, 5, 5],
        "Total Assets": [283.2, 284.7, 292.2, 300.7, 310.2],
        "Accounts Payable": [8.5, 9, 9.5, 10, 11],
        "Accrued Liabilities": [2, 2.1, 2.2, 2.3, 2.4],
        "Other Current Liabilities": [1, 0, 2, 1, 0],
        "Total Current Liabilities": [11.5, 11.1, 13.7, 13.3, 13.4],
        "Revolving Credit Facility": [
            13.3,
            18.94032587,
            28.01553129,
            37.09756706,
            40.7895349,
        ],
        "Term Loan": [180, 160, 140, 120, 100],
        "Unsecured Debt": [50, 50, 50, 50, 50],
        "Other Liabilities": [1, 2, 1, 0, 2],
        "Total Liabilities": [
            255.8,
            242.0403259,
            232.7155313,
            220.3975671,
            206.1895349,
        ],
        "Retained Earnings": [17.4, 32.65967413, 49.48446871, 70.30243294, 94.0104651],
        "Common Stock": [10, 10, 10, 10, 10],
        "Total Shareholders Equity": [
            27.4,
            42.65967413,
            59.48446871,
            80.30243294,
            104.0104651,
        ],
        "Total Liabilities and Equity": [283.2, 284.7, 292.2, 300.7, 310.2],
        "Net Income": [None, 15.25967413, 16.82479457, 20.81796423, 23.70803216],
        "Depreciation and Amortization": [None, 5, 5, 5, 5],
        "Accounts Receivable": [None, -1, -1, -1, -1],
        "Inventory": [None, -0.5, -0.5, -0.5, -0.5],
        "Other Current Assets": [None, 0, 0, 0, 0],
        "Accounts Payable": [None, 0.5, 0.5, 0.5, 1],
        "Accrued Liabilities": [None, 0.1, 0.1, 0.1, 0.1],
        "Other Current Liabilities": [None, -1, 2, -1, -1],
        "Change in Other Liabilities": [None, 1, -1, -1, 2],
        "Cash Flow from Operations": [
            None,
            19.35967413,
            21.92479457,
            22.91796423,
            29.30803216,
        ],
        "Capital Expenditures": [None, -10, -11, -12, -13],
        "Asset Dispositions": [None, 0, 0, 0, 0],
        "Cash Flow from Investing": [None, -10, -11, -12, -13],
        "Change in Unsecured Debt": [None, 0, 0, 0, 0],
        "Cash Flow from Financing": [
            None,
            -14.35967413,
            -10.92479457,
            -10.91796423,
            -16.30803216,
        ],
        "Net Cash Flow": [None, -5, 0, 0, 2.06057e-13],
        "Beginning Cash Position": [None, 5, 0, 0, 0],
        "Change in Cash Position": [None, -5, 0, 0, 2.06057e-13],
        "Ending Cash Position": [None, 0, 0, 0, 2.06057e-13],
        "Cash Flow Before Revolver": [
            None,
            -5.640325866,
            -9.075205429,
            -9.08203577,
            -3.691967835,
        ],
        "Beginning Revolver Balance": [
            None,
            13.3,
            18.94032587,
            28.01553129,
            37.09756706,
        ],
        "(Paydown) / Drawdown": [
            None,
            5.640325866,
            9.075205429,
            9.08203577,
            3.691967835,
        ],
        "Ending Revolver Balance": [
            None,
            18.94032587,
            28.01553129,
            37.09756706,
            40.7895349,
        ],
        "Interest Rate": [None, 0.06, 0.06, 0.06, 0.06],
        "Interest Expense": [None, 0.967209776, 1.408675715, 1.953392951, 2.336613059],
        "Term Loan Beginning Balance": [None, 180, 160, 140, 120],
        "(Paydown) / Drawdown": [None, -20, -20, -20, -20],
        "Term Loan Ending Balance": [None, 160, 140, 120, 100],
        "Interest Rate": [None, 0.065, 0.065, 0.065, 0.065],
        "Interest Expense": [None, 11.05, 9.75, 8.45, 7.15],
        "Unsecured Debt Beginning Balance": [None, 50, 50, 50, 50],
        "(Paydown) / Drawdown": [None, 0, 0, 0, 0],
        "Unsecured Debt Ending Balance": [None, 50, 50, 50, 50],
        "Interest Rate": [None, 0.12, 0.12, 0.12, 0.12],
        "Interest Expense": [None, 6, 6, 6, 6],
        "Total Interest Expense": [
            None,
            18.01720978,
            17.15867571,
            16.40339295,
            15.48661306,
        ],
        "Interest Earned on Cash": [None, 0, 0, 0, 0],
    }
)

In [56]:
# Apply the input data
income_statement = IncomeStatement(assumptions, historical_data)
balance_sheet = BalanceSheet(assumptions, historical_data)

In [57]:
# Get the projected data
projected_income_statement = income_statement.calculate_all_line_items()
# Print the projected data
print("Projected Income Statement:")
print(projected_income_statement)

Projected Income Statement:
   Year     Revenue  Cost of Goods Sold (COGS)  Gross Profit  SG&A Expenses  \
0  2017  210.000000                  84.000000    126.000000      63.000000   
1  2018  220.500000                  88.200000    132.300000      66.150000   
2  2019  231.525000                  92.610000    138.915000      69.457500   
3  2020  243.101250                  97.240500    145.860750      72.930375   
4  2021  255.256313                 102.102525    153.153788      76.576894   

   Operating Income  Interest Expense  Net Income  
0         63.000000          2.508000   36.295200  
1         66.150000          2.420403   38.237758  
2         69.457500          2.327155   40.278207  
3         72.930375          2.203976   42.435840  
4         76.576894          2.061895   44.708999  


In [45]:
# projected bance sheet
projected_balance_sheet = balance_sheet.calculate_all_line_items()
print("\nProjected Balance Sheet:")
print(projected_balance_sheet)


Projected Balance Sheet:
   Year  Inventory  Accounts Receivable  Other Current Assets  \
0  2017   8.284932            13.150685                     1   
1  2018   8.697945            13.972603                     1   
2  2019   9.098630            14.794521                     1   
3  2020   9.604110            15.616438                     1   
4  2021   9.863014            16.438356                     1   

   Total Current Assets  Net PP&E  Goodwill  Other Assets  Total Assets  \
0             22.435616     252.2         5             0    279.635616   
1             23.670548     257.2         5             0    285.870548   
2             24.893151     263.2         5             0    293.093151   
3             26.220548     270.2         5             0    301.420548   
4             27.301370     278.2         5             0    310.501370   

   Accounts Payable  Accrued Liabilities  Other Current Liabilities  \
0          9.205479               2.0160                     